In [1]:
from qiskit import *
import subprocess
from qiskit.tools.monitor import job_monitor
import numpy as np



In [19]:
IBMQ.load_account()
IBMQ.providers()

ibmqfactory.load_account:WARNING:2020-05-08 11:36:46,311: Credentials are already in use. The existing account in the session will be replaced.


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [20]:
provider = IBMQ.get_provider(group='open')
print(provider.backends())
device = provider.get_backend('ibmq_16_melbourne')
simulator=Aer.get_backend('qasm_simulator')
# backend=sim_backend


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]


In [21]:
simulator=Aer.get_backend('qasm_simulator')

In [22]:
import  math
from qiskit.tools.visualization import plot_histogram


def showResults(counts, nbit, bitArr):

    res_map = getResMap(counts, nbit, bitArr)
    plot_histogram(res_map)




def getResMap(counts, nbit, bitArr):
    res_map = {}
    totalSize = 1 << nbit;
    # print("total map size:", totalSize)
    for i in range(0, totalSize):
        str = ""
        for j in range(0, nbit):
            if (((i >> j) & 1) == 0):
                str = '0' + str
            else:
                str = '1' + str
        res_map[str] = 0

    for str in counts:
        m = str[::-1]
        s = ""
        for i in bitArr:
            s = s + m[i]

        res_map[s] = res_map[s] + counts[str]

    return res_map


def getPerformanceMeasure(counts, nbit, bitArr, expectedPattern):  # assumption : only single expected pattern. i.e
    res_map = getResMap(counts, nbit, bitArr)  # no superpositional output

   
    # if (res_map[expectedPattern] < secondBest):
    #     return 0
    # return res_map[expectedPattern]-secondBest
    return res_map[expectedPattern]

def getExpectedPattern(counts, nbit, bitArr):  # assumption : only single expected pattern. i.e
    res_map = getResMap(counts, nbit, bitArr)  # no superpositional output

    BestVal = -1;
    BestPattern=''
    for s in res_map:
        if ( res_map[s] > BestVal):
            BestVal = res_map[s]
            BestPattern = s ;
    # if (res_map[expectedPattern] < secondBest):
    #     return 0
    # return res_map[expectedPattern]-secondBest
    return BestPattern;





In [23]:
def run_sub_process(ckt_name,p_id,config):
    p = subprocess.run('mapper_'+str(p_id)+'m.exe '+config+' '+str(ckt_name)+' out_'+str(p_id)
                       +'.qasm layout_'+str(p_id)+'.txt')
    # print(p)
    return p.returncode

def get_a_maped_qasm(ckt_name,p_id,config):
    # print(qasm)
    # in_qasm=open('input'+str(ckt_num)+'.qasm','w')
    # in_qasm.write(qasm)
    # in_qasm.close()

    while(run_sub_process(ckt_name,p_id,config)==1):
        print(p_id,config)
        print("Try runing again")

    # out_qasm=open('out.qasm','r')

    # qasm=""
    # for line in out_qasm:
    #     qasm+=line
    #
    # out_qasm.close()
    layout={}
    time=0.0
    for line in open('layout_'+p_id+'.txt','r'):
        splits=line.split()
        if(len(splits)>1):
            q1,q2=splits
            q1=int(q1)
            q2=int(q2)
            layout[q1]=q2
        elif(len(splits)==1):
            time=splits[0]

    return layout,time




In [24]:
def is_valid_mapping(qc,prev_qc,nQBit,qBitArr,qBitArr_2):
    sim_backend=simulator
    simResult = execute(qc, backend=sim_backend, shots=2).result() 
    simResult_ex_map = execute(prev_qc, backend=sim_backend, shots=2).result()

            

    expectedPattern_ex_map=getExpectedPattern(simResult_ex_map.get_counts(prev_qc),nQBit,qBitArr)
    expectedPattern=getExpectedPattern(simResult.get_counts(qc),nQBit,qBitArr_2)
    if(expectedPattern!=expectedPattern_ex_map):
        print('expected: '+expectedPattern_ex_map)
        print('found   : '+expectedPattern)
        print('Mapping Error')
        return False
    else:
        return True

In [25]:
def extract_result(result,qc_name,qBitArr_2,nQBit,p_id,f_out,time,expectedPattern):
    counts=result.get_counts(qc_name)

#     print(counts)
#     showResults(counts,nQBit,qBitArr_2)
            
    qbits = ''

    for i in range(len(qBitArr_2)):
        if(i!=len(qBitArr_2)-1):
            qbits = qbits + str(qBitArr_2[i]) + ':'
        else :
            qbits += str(qBitArr_2[i])
            # print('8')

    expectedResult = getPerformanceMeasure(counts,nQBit,qBitArr_2,expectedPattern)
            
    f=open(f_out,'a')
    output=str(qc_name)+' '+str(p_id)+' '+str(time)+\
    ' '+qbits+' '+expectedPattern+' '+str(expectedResult)\
                       +' '+str(shots)+'\n'
    f.write(output)
            # print('11')
    print(output,'\n')

    f.close()



In [26]:
class QC:
    def __init__(self,qc, qc_name , nQBit,  qBitArr_2, p_id, ckt_name, time):
        self.qc = qc 
        self.qc_name = qc_name
        self.qBitArr_2 = qBitArr_2
        self.nQBit = nQBit 
        self.p_id = p_id
        self.ckt_name = ckt_name 
        self.time = time
    def dump(self,file_name):
        
        sim_backend=simulator
        simResult = execute(self.qc, backend=sim_backend, shots=2).result() 
        expectedPattern=getExpectedPattern(simResult.get_counts(self.qc),self.nQBit,self.qBitArr_2)

        f=open(file_name,'a')
        line=self.qc_name+' '+str(self.p_id)+' '+str(self.time)+\
        ' '+str(self.ckt_name)+' '+str(expectedPattern)+' '+str(self.nQBit)
        for q in self.qBitArr_2:
            line+=' '+str(q)
        line+='\n'
        
        f.write(line)
        f.close()

In [27]:
def get_a_mapped_circuit(nQBit,qBitArr,ckt_name,p_id):
    config="config_m.txt"
    print("running mapper # "+str(p_id))
    print("ckt_name    : "+ ckt_name)
    prev_qc=QuantumCircuit().from_qasm_file(ckt_name)
    prev_qc.barrier(range(prev_qc.num_qubits))
    prev_qc.measure(range(prev_qc.num_qubits),range(prev_qc.num_qubits))
    layout, time= get_a_maped_qasm(ckt_name,p_id,config)
    qBitArr_2 = []
    for i in range(nQBit):
        qBitArr_2.append(layout[qBitArr[i]])
    print(qBitArr)
    print(qBitArr_2)
    
    out_ckt_name = 'out_'+p_id+'.qasm'
    qc=QuantumCircuit().from_qasm_file(out_ckt_name)
    qc.barrier(range(qc.num_qubits))
    qc.measure(range(qc.num_qubits),range(qc.num_qubits))
    
    if( is_valid_mapping(qc,prev_qc,nQBit,qBitArr,qBitArr_2)==False):
        
        return None, None, None
    else:
        return qc, qBitArr_2, time 
    

In [28]:
def get_a_mapped_QC(nQBit,qBitArr,ckt_name,p_id,name):
    qc, qBitArr_2, time = get_a_mapped_circuit(nQBit, qBitArr, ckt_name, p_id)
#     print(qc)
    if(qc is None):
#         print(qc)
        return None
    else:
        qc_name=name+'_'+str(p_id)
        qc.name= qc_name 
#         print(qc)
        return QC(qc, qc_name , nQBit,  qBitArr_2, p_id, ckt_name, time)
    

In [29]:
def generate_all_circuits(ckt_map, p_id_list):
    QC_list = []
    for k in ckt_map:
        name = k 
        nQBit = ckt_map[k]
        ckt_name = name+'.qasm'
        qBitArr= range(nQBit)
        for p_id in p_id_list:
            nQC= get_a_mapped_QC(nQBit,qBitArr,ckt_name,str(p_id),name)
            if(nQC == None ):
                continue 
            else:
                QC_list.append(nQC)
            
    return QC_list 
            
    

In [30]:
def save_experiment(job_id, QC_list, exp_file):
    f=open(exp_file,'w')
    f.write(str(job_id)+'\n')
    size=len(QC_list);
    f.write(str(size)+'\n')
    
    f.close()
#     print(exp_file)
    for q in QC_list:
#         print(q.qc_name)
#         print(exp_file)
        q.dump(exp_file)
        
    

In [42]:
def load_experiment_result(exp_file,result_file,backend):
    exp_file=open(exp_file,'r')
    job_id=str(exp_file.readline()).strip()
    total_circuits= int(exp_file.readline())
    print(job_id)
    print(total_circuits)
    result=backend.retrieve_job(job_id).result()
    print('job_id: ' + job_id)
    print('total_ckts: '+ str(total_circuits))
    
    for i in range(total_circuits):
        line= exp_file.readline().split()
        qc_name=line[0]
        p_id   =line[1]
        time   =line[2]
        ckt_name=line[3]
        expectedPattern = line[4]
        nQBit = int(line[5])
        qBitArr_2=[]
        for k in range(nQBit):
            qBitArr_2.append(int(line[6+k]))
            
#         print('qBitArr_2: ')
#         print(qBitArr_2)
        extract_result(result,qc_name,qBitArr_2,nQBit,p_id,result_file,time,expectedPattern)

    

In [43]:
def load_experiment_result_local(exp_file,result_file,backend,job):
    exp_file=open(exp_file,'r')
    job_id=str(exp_file.readline()).strip()
    total_circuits= int(exp_file.readline())
    result=job.result()
    print('job_id: ' + job_id)
    print('total_ckts: '+ str(total_circuits))
    
    for i in range(total_circuits):
        line= exp_file.readline().split()
        qc_name=line[0]
        p_id   =line[1]
        time   =line[2]
        ckt_name=line[3]
        expectedPattern = line[4]
        nQBit = int(line[5])
        qBitArr_2=[]
        for k in range(nQBit):
            qBitArr_2.append(int(line[6+k]))
            
#         print('qBitArr_2: ')
#         print(qBitArr_2)
               
        extract_result(result,qc_name,qBitArr_2,nQBit,p_id,result_file,time,expectedPattern)

    

In [44]:
def run_experiment(QC_list,backend,shots):
    qc_list=[]
    for q in QC_list:
        qc_list.append(q.qc)
    
    job=execute(qc_list, backend=backend,shots=shots)
    
    return job

In [36]:
import datetime as dt
np.random.seed(77777)
ckt_map={'Adder':4,
         'BV4'  :4,
         'BV4_1111':4,
         'BV6':6,
         'BV6_111111':6 ,
         'BV8': 8,
         'Fredkin' :3,
         'HS2' : 2,
         'HS4' : 4,
         'HS6' : 6,
         'Or'  : 3,
         'Peres': 3,
         'Shor' :4,
         'Toffoli':3 
        }
ckt_map={'Toffoli':3} # debug
p_id_list=[1,2]
shots=8192

exp_file=input('give experiment file name: ')
if(exp_file=='t'):
    exp_file='exp_file_'+str(dt.date.today())+'.txt'
else:
    exp_file='exp_'+exp_file+'.txt'

result_file=input('give result file name: ')
if(result_file=='t'):
    result_file='result_file_'+str(dt.date.today())+'.txt'
else:
    result_file='res_'+result_file+'.txt'



backend=input('backend: d or s: ')
if(backend=='d'):
    backend=device
else:
    backend=simulator
    
QC_list=generate_all_circuits(ckt_map,p_id_list)
job = run_experiment(QC_list,backend,shots)
print('job_id: '+job.job_id())
save_experiment(job.job_id(), QC_list, exp_file)


job_monitor(job)
print(exp_file)
print(result_file)

print(backend)


# load_experiment_result(exp_file,result_file,backend)

load_experiment_result_local(exp_file,result_file,backend,job)


give experiment file name: toffoli
give result file name: toffoli
backend: d or s: d
running mapper # 1
ckt_name    : Toffoli.qasm
range(0, 3)
[3, 4, 11]
running mapper # 2
ckt_name    : Toffoli.qasm
range(0, 3)
[4, 10, 11]
job_id: 5eb4f6eae1699a0011a74b0b
Job Status: job has successfully run
exp_toffoli.txt
res_toffoli.txt
ibmq_16_melbourne
job_id: 5eb4f6eae1699a0011a74b0b

total_ckts: 2
Toffoli_1 1 0.001 3:4:11 000 6720 8192
 

Toffoli_2 2 0.003 4:10:11 000 5388 8192
 



In [40]:
load_experiment_result_local(exp_file='exp_toffoli.txt',result_file='res_toffoli.txt',backend,job)

FileNotFoundError: [Errno 2] No such file or directory: 'exp_file_2020-5-8.txt'

In [45]:
exp_file='exp_file_2020-05-08.txt'
result_file='res_file_2020-05-08.txt'
load_experiment_result(exp_file,result_file,backend)


5eb4f049746cbd00182a45bb
28
job_id: 5eb4f049746cbd00182a45bb
total_ckts: 28
Adder_1 1 0.014 10:11:3:4 0000 6363 8192
 

Adder_2 2 0.001 13:9:0:1 0000 1657 8192
 

BV4_1 1 0 0:4:11:2 0111 5365 8192
 

BV4_2 2 0.001 1:4:0:14 0111 878 8192
 

BV4_1111_1 1 0.002 3:11:2:10 1111 2842 8192
 

BV4_1111_2 2 0.002 4:0:14:1 1111 586 8192
 

BV6_1 1 0.002 5:1:0:4:11:2 000111 4907 8192
 

BV6_2 2 0.002 1:2:3:4:0:14 000111 936 8192
 

BV6_111111_1 1 0.008 4:11:2:3:5:9 111111 325 8192
 

BV6_111111_2 2 0.001 4:0:14:1:13:7 111111 143 8192
 

BV8_1 1 0 7:6:5:1:0:4:11:2 00000111 4925 8192
 

BV8_2 2 0.002 1:2:3:5:6:4:0:14 00000111 927 8192
 

Fredkin_1 1 0.007 11:3:4 000 4212 8192
 

Fredkin_2 2 0.002 0:2:1 000 3447 8192
 

HS2_1 1 0.002 4:3 11 6662 8192
 

HS2_2 2 0.001 4:10 11 6801 8192
 

HS4_1 1 0.001 4:3:10:11 1011 6150 8192
 

HS4_2 2 0.001 4:10:1:2 1011 4597 8192
 

HS6_1 1 0.003 4:3:2:12:10:11 101100 2679 8192
 

HS6_2 2 0.001 4:10:1:2:0:14 101100 4323 8192
 

Or_1 1 0.004 3:4:11 000 5613 8192
 